# 섹션 0 - OT
- 참고자료: Python을 이용한 개인화 추천시스템
- 강의 목적: 주요 개인화 추천 알고리즘의 작동원리 이해하기
- 강의에서 주로 다룰 내용
    - 개인화 추천 기술의 전반적인 개념
    - 협업 필터링
    - 행렬 요인화
    - 딥러닝 추천 알고리즘
    - 하이브리드 추천 알고리즘


# 섹션 1 - 주요 추천 알고리즘
- 과거: 각 집단에 잘 맞는 제품이나 서비스 추천
- 현재: 개개인에게 맞는 제품이나 서비스 추천(개개인이 집단 그 자체)
- 추천시스템의 여러 기술: 협업필터링, 내용 기반 필터링, 지식 기반 필터링, 딥러닝, 하이브리드 필터링(헙업필터링 & 딥러닝)

## 1.1 주요 추천 알고리즘
1. 협업 필터링(Collaborative Filtering: CF): 평가 패턴이 비슷한 집단 속에서 서로 접하지 않은 제품을 추천하는 기술
- 한계: 소비자들의 평가 정보가 없는 경우 ex) 신규 고객, 휴면 고객
- 해결책: 쇼핑몰 특정 페이지 체류 시간, 클릭 등 간접적인 데이터를 이용한다.
2. 내용 기반 필터링(Content-Based Filtering: CB): 제품의 내용을 분석해서 추천하는 기술. 해당 제품의 메타데이터를 기술할 수 있다면, 유사한 제품을 추천할 수 있다. 뉴스나 책 같은 것을 추천할 때 많이 사용된다.

3. 지식 기반 필터링(Knowledge-Based Filtering: KB): 특정 분야 전문가의 도움을 받아서 그 분야에 대한 '전체적인 지식 구조(온톨로지, **체계도**)'를 만들어서 활용하는 방법
- 협업, 내용 기반 필터링의 단점은 전체적인 구조도가 없다는 점이다. 어떤 제품을 좋아하는지 분석은 가능하지만, 왜 그 제품을 좋아하는 지를 알 수 없다. 반면, KB에서는 왜 그 제품을 좋아하는 지 알 수 있다.
- 단점: 모든 분야에 전문가가 있어야 한다.
4. 딥러닝(Deep Learning: DL)
- 장점: 예측도가 높다. 다양한 입력변수(이미지, 음성 등)를 사용할 수 있다.
5. 하이브리드(Hybrid) 기술: 두 가지 이상의 알고리즘을 혼합한 형태

## 1.2 추천 시스템 적용 사례
1. 넷플리스

2. 아마존
- 협업 필터링, 하이브리드 추천 기술 사용
- 사용자 제품 평가 데이터를 다양하게 수집한다. 간접 데이터 수집.
    - 고객 평가
    - 상품 페이지 방문
    - 체류 시간
    - 쇼핑 카트 분석

# 섹션 2 - 기본적인 추천 시스템
- MovieLens 데이터
    - 영화 1점(최악) ~ 5점(최고) 평가
    - MovieLens 100k(10만개)와 20M(2000만개) 사용

## 2.1 데이터 읽기
- MovieLense 100K 데이터는 3가지 파일로 구성됨
    - 사용자 데이터: u.user
    - 영화에 대한 데이터: u.item
    - 영화 평가 데이터: u.data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 사용자 u.user 파일을 DataFrame으로 읽기
import os
import pandas as pd

base_src = 'drive/MyDrive/강의/RecoSys/Data'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep='|',
                    names=u_cols,
                    encoding='latin-1')

users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [3]:
# 사용자 u.item 파일을 DataFrame으로 읽기
u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release date', 'video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src,
                     sep='|',
                     names=i_cols,
                     encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
# 사용자 u.data 파일을 DataFrame으로 읽기
u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src,
                      sep='\t',
                      names=r_cols,
                      encoding='latin-1')
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


## 2.2 인기제품 방식
- 개별 사용자에 대한 정보가 너무 없거나, 간단한 추천을 제공하기 위해서 사용한다.
- 모든 사용자들에게 동일한 상품(인기있는 상품 즉, Best-Seller 제품)을 추천한다.
- 인기 제품을 추천하기 위해서는 각 제품에 대한 평가의 평균을 구해서 가장 높은 순서대로 추천하면 된다.

In [5]:
# 인기 제품 추천 방식 function
def recom_movie(n_items): # 인자 n_items는 몇 개의 아이템을 추천할지에 대한 인자임
    movie_mean = ratings.groupby(['movie_id'])['rating'].mean() # movie_id를 기준으로 rating의 평균을 낸다.
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items] # 평균 낸 것을 정렬하고, 몇 개를 뽑아낼 건지 정한다.
    recom_movies = movies.loc[movie_sort.index] # 뽑힌 movie_sort의 index를 기준으로 전체 movie 데이터에서 조회함.
    recommendations = recom_movies['title'] # 추천된 영화의 title을 뽑아낸다.
    return recommendations

recom_movie(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

## 2.3 추천 시스템의 정확도 측정
- 추천 시스템의 성능 = "정확성"
    - 정확성 외에 다양한 지표가 있지만, 일단은 정확성에 대해 배운다.
- 어떤 추천시스템이 매우 정확하다는 말의 의미는, 추천시스템이 추천한 예측 선호도와 실제 선호도 간에 차이가 별로 없어야 한다는 의미이다.
    - A가 실제로 4.5점의 평점을 준 영화에 대해 추천시스템이 예측한 평점이 2점이라면 좋지 않은 모델이다.
- 추천시스템 또한 다른 머신러닝 모델들과 같이 데이터를 훈련 데이터, 테스트 데이터, (검증 데이터)로 나눈다.
- RMSE(Root Mean Squared Error): (참값 - 예측값)^2 의 전체 합을 구한 다음에 데이터 셋 숫자(n) 으로 나눠준 뒤 루트를 씌워준다.

In [6]:
# 100k의 영화 평점에 대해서 실제값과 best-seller 방식으로 구한 예측값의 RMSE를 계산하는 코드
import numpy as np

def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 정확도 계산
rmse = []
movie_mean = ratings.groupby(['movie_id'])['rating'].mean() # bset_seller 방식으로 계산

for user in set(ratings.index): # 영화를 평가한 user_id가 중복되지 않도록 집합으로 만듦
    y_true = ratings.loc[user]['rating'] # 사용자가 평가한 모든 영화의 평점을 저장
    # best-seller 방식으로
    y_pred = movie_mean[ratings.loc[user]['movie_id']] # movie_id 에 대한 예측값
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy) # rmse 결과값을 빈 리스트에 넣는다.

# RMSE 계산
print(np.mean(rmse))

0.996007224010567


## 2.4 사용자 집단별 추천
- 위에서 한 best-seller 방식은 전체 사용자의 평점 평균을 사용했다. 그런데 집단 간 평가 경향이 있는 경우에는 노이즈 값이 많이 낄 수 있다. 예를 들어, 남성 여성에 따라 좋아하는 영화 장르가 있을 수 있다.
- 남성, 여성으로 집단을 나눠서 best-seller 방식을 사용한다.

In [7]:
# 데이터 읽기
base_src = 'drive/MyDrive/강의/RecoSys/Data'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep='|',
                    names=u_cols,
                    encoding='latin-1')


u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id', 'title', 'release date', 'video release date',
          'IMDB URL', 'unknown', 'Action', 'Adventure', 'Animation',
          'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv(u_item_src,
                     sep='|',
                     names=i_cols,
                     encoding='latin-1')


u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src,
                      sep='\t',
                      names=r_cols,
                      encoding='latin-1')


# 이번 실습에 맞게 데이터 읽기 코드 추가
ratings = ratings.drop(['timestamp'], axis=1)
movies = movies[['movie_id', 'title']]

In [8]:
# 데이터 train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy() # 원본 데이터 유지를 위해 copy
y = ratings['user_id'] # ratings 에 있는 user_id 를 할당함

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.25,
                                                    stratify=y) # 층화추출. 데이터마다 라벨값이 많고 적은 숫자가 다르다. 데이터를 추출할 때 아예 데이터가 안 뽑히는 경우가 생길 수 있기에 그것을 방지하고자 원천 데이터에 있는 y의 환경을 train, test set에도 반영하는 옵션이다.

# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수
def score(model): # 내가 만든 model의 성능을 test set으로 검증하는 함수
    id_pairs = zip(x_test['user_id'], x_test['movie_id']) # 두 값의 pair를 맞춰서 tuple형 변수에 할당함.
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs]) # 모든 사용자, 영화 pair에 대해서 주어진 예측 모델에 예측값을 계산해 최종적으로 np.array 형태로 y_pred에 할당함
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# best_seller 함수를 이용한 정확도 계산
train_mean = x_train.groupby(['movie_id'])['rating'].mean()
def best_seller(user_id, movie_id):
    # 전체 데이터를 다 넣을 때는 영화가 무조건 있지만, train, test 데이터로 나눴기 때문에 index error 가 발생할 수 있다.
    # 해당 영화가 평균 데이터에 존재하면 평균값을 돌려주고, 존재하지 않으면 기본값 3.0을 준다.
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

score(best_seller)
# 성능이 더 낮게 나왔다. 지난 시간에는 전체 데이터로 학습하고 전체 데이터로 평가했기 때문에 성능이 더 잘 나왔던 것이다.

1.016970021381184

*참고)* **층화추출**: 분류 문제에서 클래스 A와 클래스 B가 각각 90개와 10개의 총 100개의 데이터가 있을 경우, stratify=y로 설정하면 학습용 데이터와 테스트용 데이터에서 클래스 A와 클래스 B가 각각 9:1의 비율을 유지하도록 데이터를 분할합니다. 즉, 학습용 데이터와 테스트용 데이터에서도 클래스 비율이 동일하게 유지됩니다.

In [9]:
# 참고) id_pairs 값 찍어보기
id_pairs = zip(x_test['user_id'], x_test['movie_id'])
print(list(id_pairs))

[(707, 9), (613, 89), (487, 402), (490, 237), (3, 271), (270, 736), (184, 64), (416, 476), (709, 38), (478, 168), (44, 678), (637, 289), (904, 794), (18, 202), (181, 879), (441, 121), (705, 300), (551, 603), (385, 122), (206, 269), (425, 218), (466, 273), (514, 153), (249, 174), (567, 387), (296, 10), (320, 4), (82, 211), (493, 288), (450, 47), (453, 471), (883, 582), (883, 847), (397, 615), (790, 755), (653, 448), (435, 1039), (352, 568), (474, 70), (457, 276), (551, 1079), (164, 299), (546, 413), (151, 52), (486, 1302), (445, 276), (757, 742), (918, 1639), (497, 472), (29, 245), (437, 702), (472, 365), (22, 210), (60, 205), (880, 720), (933, 834), (130, 174), (18, 461), (416, 385), (399, 215), (789, 762), (275, 89), (204, 1296), (933, 403), (314, 932), (233, 82), (774, 436), (883, 22), (488, 136), (303, 1510), (463, 310), (239, 921), (429, 222), (299, 213), (328, 637), (276, 288), (401, 481), (878, 655), (758, 122), (308, 199), (639, 863), (774, 1079), (854, 493), (497, 549), (406, 1

In [10]:
# 성별에 따른 예측값 계산
merged_ratings = pd.merge(x_train, users)

users = users.set_index('user_id')

g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean() # 성별에 따른 평점 평균

rating_matrix = x_train.pivot(index='user_id',
                              columns='movie_id',
                              values='rating')

rating_matrix # train 데이터로 만든 pivot 테이블

movie_id,1,2,3,4,5,6,7,8,9,10,...,1671,1672,1673,1674,1675,1676,1678,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,4.0,3.0,3.0,NaN,NaN,NaN,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Gender 기준 추천
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix.columns: # 인자값으로 넣은 movie_id 가 rating_matrix 의 column 값에 있으면
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id].index: # 예측 대상 영화가 남성, 여성 별로 다 있는 건지 확인한다.
            gender_rating = g_mean[movie_id][gender] # 있다면 movie_id에 대한 gender 평균 값을 낸다.
        else:
            gender_rating = 3.0 # 없으면 기본값으로 3.0을 준다.
    else:
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

1.0246329948186879

-> 결론: 그룹별로 성능 차이를 시행착오를 통해 알아내는 시도를 해보는 것이 모델 성능 향상에 도움이 된다.